#Extracción de información usando métodos estadísticos

## Named Entity Recognition and Classification

Objetivo: Identificar entidades nombradas de los tipos principales como Personas, Lugares u Organizaciones 

Ejemplo sencillo de como realizar una tarea de clasificación secuencial

* Codificación de características 
* Uso de algoritmos de clasificación 
* Evaluación 
* Ventajas e inconvenientes


In [ ]:
import nltk

from nltk.corpus import conll2002

print conll2002.readme()

## Inspección del corpus

Comentar las características especiales del corpus
* Conjunto de entrenamiento, test y development 
* Estructura tabular: 
    * Token POS_tag NE_tag
    * Codificación de los chucks: IOB2 format: B-PER I-PER O
    * Clases: 
    * Separación de frases y documentos 
    * Todos los documentos van a uno de los conjuntos de datos - mantener la coherencia - ¿Cómo hacer validación cruzada?
* Frecuencia de las categorias 
* Evaluacion

In [ ]:
conll2002?

In [ ]:
# Formato del corpus en modo texto 

conll2002.raw()[0:200]

In [ ]:
# ¿Como representa la informacion el corpus en Python NLTLK

words = conll2002.words()
tagged_words = conll2002.tagged_words()
sents = conll2002.sents()
chunked_sents = conll2002.chunked_sents()

# Does not support the para() - no hay información de parrafos

print words

In [ ]:
for tree in chunked_sents[4:8]:
    print(tree) 

### Formato IOB

In [ ]:
iob_sentences = conll2002.iob_sents() 

print iob_sentences

In [ ]:
[(word, tag) for (word, tag, ne_tag) in iob_sentences[2]]

### Acceso a los diferentes subconjuntos de datos 

In [ ]:
# Acceso al conjunto de entrenamiento para castellano en formato iob 
trainSentences = conll2002.iob_sents('esp.train')

print trainSentences

# Acceso al conjunto de test para castellano como listas de oraciones  
testSentences = conll2002.sents('esp.testa')

print testSentences

## Ejemplo de uso que deseamos

```python
# Instanciar el tagger 
# - Usa el conjunto de entrenamiento etiquetado
# - Extraer características
# - Entrenar el modelo
train_sentences = conll2002.iob_sents('esp.train')
nerctagger = NamedEntityTagger(train_sentences)

testSentences = conll2002.iob_sents('esp.testa')
test_sentence = [(word,tag) for (word,tag, ne_tag) in test_sentences[107]]

# Etiquetar oraciones con la tercera columna (ne_tag) dadas las otras dos (word, tag)
# - Extraer características 
# - Usar el modelo entrenado para inferir la nueva etiqueta
# - Otros pasos?
nerctagger.tagger(test_sentence)

# Evaluación: Comparar las etiquetas de nuestro nerctagger con las del gold standard 
# Uso de precision y recall
```